<a href="https://colab.research.google.com/github/ssosoo/2024_DS60/blob/main/%EB%B2%A0%EB%A5%B4%EB%88%84%EC%9D%B4_%EB%82%98%EC%9D%B4%EB%B8%8C_%EB%B2%A0%EC%9D%B4%EC%A6%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

np.random.seed(5)

In [2]:
email_list = [
                {'email title': 'free game only today', 'spam': True},
                {'email title': 'cheapest flight deal', 'spam': True},
                {'email title': 'limited time offer only today only today', 'spam': True},
                {'email title': 'today meeting schedule', 'spam': False},
                {'email title': 'your flight schedule attached', 'spam': False},
                {'email title': 'your credit card statement', 'spam': False}
             ]
df = pd.DataFrame(email_list)

In [3]:
df

,email title,spam
0,free game only today,True
1,cheapest flight deal,True
2,limited time offer only today only today,True
3,today meeting schedule,False
4,your flight schedule attached,False
5,your credit card statement,False


데이터 다듬기

제목으로 학습 진행하고, 레이블 0,1값으로 스팸 메일 여부 판단한다.

이메일 제목에 특정 단어가 출현할 경우 무조건 1을, 없을 경우엔ㄴ 0을 갖도록 설정합니다.

In [4]:
df['label'] = df['spam'].map({True:1, False:0})

df_x=df["email title"]
df_y=df["label"]


CountVectorizer을 사용할 때, binary=True를 사용해 자주 등장하는 단어를 1로 처리한다.
this, is같은 불용어는 제외.

In [5]:
#벡터화
cv = CountVectorizer(binary=True)
x_traincv = cv.fit_transform(df_x)

In [6]:
encoded_input=x_traincv.toarray()
encoded_input

array([[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]])

In [7]:
#2차원화 적용해 인덱스 확인
cv.inverse_transform(encoded_input[0].reshape(1,-1))


[array(['free', 'game', 'only', 'today'], dtype='<U9')]

In [9]:
#인덱스 확인법
cv.get_feature_names_out()

array(['attached', 'card', 'cheapest', 'credit', 'deal', 'flight', 'free',
       'game', 'limited', 'meeting', 'offer', 'only', 'schedule',
       'statement', 'time', 'today', 'your'], dtype=object)

**베르누이 나이브 베이즈 모델 학습**

In [10]:
bnb = BernoulliNB()
y_train = df_y.astype('int')
bnb.fit(x_traincv, y_train)

BernoulliNB()

**테스트 데이터 다듬기**

In [14]:
test_email_list = [
               {'email title': 'free flight offer', 'spam': True},
                {'email title': 'hey traveler free flight deal', 'spam': True},
                {'email title': 'limited free game offer', 'spam': True},
                {'email title': 'today flight schedule', 'spam': False},
                {'email title': 'your credit card attached', 'spam': False},
                {'email title': 'free credit card offer only today', 'spam': False}
             ]
test_df = pd.DataFrame(test_email_list)
test_df['label']=test_df['spam'].map({True:1, False:0})
test_x = test_df["email title"]
test_y = test_df["label"]
x_testcv = cv.transform(test_x)

In [15]:
predictions = bnb.predict(x_testcv)
accuracy_score(test_y, predictions)

0.8333333333333334